In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

In [2]:
"""Data Cleaning movie list

1. Extract movies that are only classified as either short, movie or tv Movie.
2. Remove adult films
3. Replace '\\N' with 'NA' as that is considered missing.
4. Extract movies that started from 2000 to 2021
5. Remove isAdult and 'endYear' columns as those are irrelevant. """

# read in data downloaded from https://datasets.imdbws.com/
imdb_movies = pd.read_csv("title.basics.tsv", sep = '\t', low_memory=False)

imdb_titles = imdb_movies[(imdb_movies['titleType'] == 'movie') | \
						  (imdb_movies['titleType'] == 'tvMovie') | \
						  (imdb_movies['titleType'] == 'short')]
	

imdb_titles = imdb_titles[(imdb_titles['startYear'] >= '2000') & \
						  (imdb_titles['startYear'] <= '2021')]

imdb_titles = imdb_titles[imdb_titles['isAdult'] == '0']

imdb_titles = imdb_titles.replace(to_replace = '\\N', value = 'NA')

imdb_titles = imdb_titles.drop(['isAdult', 'endYear'], axis = 1)

In [3]:
imdb_titles

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
11059,tt0011216,movie,Spanish Fiesta,La fête espagnole,2019,67,Drama
11433,tt0011594,movie,Pour don Carlos,Pour don Carlos,2021,NA,NA
11636,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019,NA,"Action,Crime"
15179,tt0015414,movie,La tierra de los toros,La tierra de los toros,2000,60,NA
33805,tt0034413,short,Youth Gets a Break,Youth Gets a Break,2001,20,Short
...,...,...,...,...,...,...,...
8650501,tt9916730,movie,6 Gunn,6 Gunn,2017,116,NA
8650511,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013,49,Documentary
8650512,tt9916756,short,Pretty Pretty Black Girl,Pretty Pretty Black Girl,2019,NA,Short
8650516,tt9916764,short,38,38,2018,NA,Short


In [4]:
title_ratings = pd.read_csv("title.ratings.tsv", sep = '\t', low_memory=False)

In [5]:

pysqldf = lambda q: sqldf(q, globals()) # using SQL in python

# query to join the title table with the ratings table which has
# average IMDB rating and number of votes on the movie

q = """SELECT t.*, r.averageRating, r.numVotes
FROM imdb_titles t
LEFT JOIN title_ratings r
ON t.tconst = r.tconst
"""

In [6]:
movie_and_ratings = pysqldf(q)

In [7]:
movie_and_ratings

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,2019,67,Drama,6.9,30.0
1,tt0011594,movie,Pour don Carlos,Pour don Carlos,2021,NA,NA,5.9,21.0
2,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019,NA,"Action,Crime",NaN,NaN
3,tt0015414,movie,La tierra de los toros,La tierra de los toros,2000,60,NA,5.3,15.0
4,tt0034413,short,Youth Gets a Break,Youth Gets a Break,2001,20,Short,NaN,NaN
...,...,...,...,...,...,...,...,...,...
950546,tt9916730,movie,6 Gunn,6 Gunn,2017,116,NA,NaN,NaN
950547,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013,49,Documentary,NaN,NaN
950548,tt9916756,short,Pretty Pretty Black Girl,Pretty Pretty Black Girl,2019,NA,Short,NaN,NaN
950549,tt9916764,short,38,38,2018,NA,Short,NaN,NaN


In [8]:
movie_and_ratings = movie_and_ratings.replace(np.nan, 0)

In [9]:
movie_and_ratings

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,2019,67,Drama,6.9,30.0
1,tt0011594,movie,Pour don Carlos,Pour don Carlos,2021,NA,NA,5.9,21.0
2,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019,NA,"Action,Crime",0.0,0.0
3,tt0015414,movie,La tierra de los toros,La tierra de los toros,2000,60,NA,5.3,15.0
4,tt0034413,short,Youth Gets a Break,Youth Gets a Break,2001,20,Short,0.0,0.0
...,...,...,...,...,...,...,...,...,...
950546,tt9916730,movie,6 Gunn,6 Gunn,2017,116,NA,0.0,0.0
950547,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013,49,Documentary,0.0,0.0
950548,tt9916756,short,Pretty Pretty Black Girl,Pretty Pretty Black Girl,2019,NA,Short,0.0,0.0
950549,tt9916764,short,38,38,2018,NA,Short,0.0,0.0


In [10]:
movie_and_ratings = movie_and_ratings.replace(np.nan, 0)

In [11]:
movie_and_ratings

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,2019,67,Drama,6.9,30.0
1,tt0011594,movie,Pour don Carlos,Pour don Carlos,2021,NA,NA,5.9,21.0
2,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019,NA,"Action,Crime",0.0,0.0
3,tt0015414,movie,La tierra de los toros,La tierra de los toros,2000,60,NA,5.3,15.0
4,tt0034413,short,Youth Gets a Break,Youth Gets a Break,2001,20,Short,0.0,0.0
...,...,...,...,...,...,...,...,...,...
950546,tt9916730,movie,6 Gunn,6 Gunn,2017,116,NA,0.0,0.0
950547,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013,49,Documentary,0.0,0.0
950548,tt9916756,short,Pretty Pretty Black Girl,Pretty Pretty Black Girl,2019,NA,Short,0.0,0.0
950549,tt9916764,short,38,38,2018,NA,Short,0.0,0.0


In [12]:
q2 = """SELECT t.*, r.averageRating, r.numVotes
FROM imdb_titles t
INNER JOIN title_ratings r
ON t.tconst = r.tconst"""
movie_and_ratings2 = pysqldf(q2)
movie_and_ratings2

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,2019,67,Drama,6.9,30
1,tt0011594,movie,Pour don Carlos,Pour don Carlos,2021,NA,NA,5.9,21
2,tt0015414,movie,La tierra de los toros,La tierra de los toros,2000,60,NA,5.3,15
3,tt0035423,movie,Kate & Leopold,Kate & Leopold,2001,118,"Comedy,Fantasy,Romance",6.4,82687
4,tt0036177,movie,Muhomatsu no issho,Muhomatsu no issho,2008,100,"Action,Adventure",7.3,118
...,...,...,...,...,...,...,...,...,...
277772,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,2019,NA,"Adventure,History,War",3.8,14
277773,tt9916460,tvMovie,Pink Taxi,Pink Taxi,2019,NA,Comedy,9.3,17
277774,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama,8.3,6
277775,tt9916544,short,My Sweet Prince,My Sweet Prince,2019,12,"Drama,Short",7.0,50
